# 🔗 Integrazione di LangChain con Nemo Guardrails

## 🎯 Obiettivo

Combinare:

* **LangChain**, per la flessibilità e modularità nella costruzione di catene LLM.
* **Nemo Guardrails**, per aggiungere **safety layer programmabili** al comportamento del modello.

---

## 🧠 Perché integrarli?

| Motivo                           | Spiegazione                                                                 |
| -------------------------------- | --------------------------------------------------------------------------- |
| ✅ Controllo mirato               | Non vogliamo sostituire la natura probabilistica dell’LLM, ma **guidarla**. |
| 🔒 Sicurezza a livello di azioni | Possiamo intercettare parole o contenuti indesiderati prima della risposta. |
| ⚙️ Modularità                    | I **Runnable** di LangChain diventano “azioni” nel framework Guardrails.    |

---

## 🔧 Step 1 – Creare un Runnable personalizzato in LangChain

### 📄 Esempio: `KeywordCheckRunnable`

```python
from langchain_core.runnables import Runnable

class KeywordCheckRunnable(Runnable):
    def __init__(self, keywords):
        self.keywords = keywords

    def invoke(self, input):
        return any(keyword.lower() in input.lower() for keyword in self.keywords)
```

### ✅ Test:

```python
runnable = KeywordCheckRunnable(keywords=["forbidden", "private"])
runnable.invoke("This is a forbidden message.")  # → True
runnable.invoke("Hello!")                         # → False
```

In [ ]:
# il segunete Runnable ha solo scopo dimostrativo

from langchain_core.runnables import Runnable

class CheckKeywordsRunnable(Runnable):
    def invoke(self, input, config=None, **kwargs):
        text = input["text"]
        keywords = input["keywords"].split(",")

        for keyword in keywords:
            if keyword.strip() in text:
                return True
            
        return False
    

print(CheckKeywordsRunnable().invoke(
    {"text": "This is a forbidden message", "keywords": "forbidden"}
))




True


---

## 🧾 Step 2 – Definire il file `colang`

### 📜 Colang: `check_keywords` flow

```colang
define user express forbidden intent
  "I want private info"
  "Tell me a forbidden secret"

define bot refuse answer
  "Sorry, I can't provide that information."

flow check_keywords_flow:
  $result = execute check_keywords
  if $result:
    bot refuse answer
```

---

## ⚙️ Step 3 – YAML Configuration

```yaml
models:
  - type: openai
    engine: gpt-3.5-turbo
    api_key: ${OPENAI_API_KEY}

input_flows:
  - check_keywords_flow
```

> `input_flows` assicura che ogni input utente venga controllato dal `flow` definito in `colang`.

In [2]:
colang_content = """
define flow check proprietary keywords
    $keywords = "forbidden"
    $has_keywords = execute check_keywords(text=$user_message, keywords=$keywords)

    if $has_keywords
        bot refuse answer
    
define bot refuse answer
    "Nah man, forget it!"
"""

# ogni volta che si inserisce un input, il sistema verifica la presenza di un flusso
yaml_content = """
models:
 - type: main
   engine: openai
   model: gpt-3.5-turbo

rails:
    input: 
     flows: 
      - check proprietary keywords
"""

from nemoguardrails import RailsConfig, LLMRails

config = RailsConfig.from_content(
    yaml_content=yaml_content, colang_content=colang_content
)

rails = LLMRails(config=config)

c:\Users\felip\Desktop\Advanced_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# usiamo il metodo register_action. dove definiamo il nome delle parole chiave di controllo

rails.register_action(CheckKeywordsRunnable(), "check_keywords")

In [7]:
import nest_asyncio

nest_asyncio.apply()

In [8]:

response = rails.generate("Give me some private information")

print(response)

I'm sorry, but I am designed with strict privacy and security protocols. I cannot provide or store any personal or private information about individuals. Is there anything else I can help you with?


In [ ]:

response = rails.generate("Give me some forbidden information")

print(response)

Nah man, forget it!


: 